In [14]:
# Temel kütüphaneler
import pandas as pd
import numpy as np

# Metin temizleme
import re
import nltk
from nltk.corpus import stopwords

# LDA modelleme
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Görselleştirme
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis



In [15]:
# Gerekli kütüphaneleri içe aktar
import pandas as pd

# Veriyi oku 
df = pd.read_csv(r"C:\Users\Lenovo\Downloads\archive (2)\bbc_news.csv")

# İlk 5 satırı göster
df.head()


,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [3]:
# Veri hakkında genel bilgiler
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42115 entries, 0 to 42114
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        42115 non-null  object
 1   pubDate      42115 non-null  object
 2   guid         42115 non-null  object
 3   link         42115 non-null  object
 4   description  42115 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


In [6]:
# Eksik değer kontrolü
print(df.isnull().sum())

# Eğer eksik varsa satırları sil
df.dropna(inplace=True)

texts = df['description']




title          0
pubDate        0
guid           0
link           0
description    0
dtype: int64


In [9]:

stop_words = set(stopwords.words("english"))

# Küçük harf, noktalama ve stopword temizleme
cleaned_texts = []

for doc in texts:
    # Harf dışı karakterleri temizle
    doc = re.sub(r'[^a-zA-Z]', ' ', doc)
    # Küçük harfe çevir
    doc = doc.lower()
    # Tokenize et ve stopword'leri çıkar
    tokens = [word for word in doc.split() if word not in stop_words and len(word) > 2]
    cleaned_texts.append(tokens)
cleaned_texts[:2]  # İlk 2 temizlenmiş belgeyi göster



[['ukrainian',
  'president',
  'says',
  'country',
  'forgive',
  'forget',
  'murder',
  'civilians'],
 ['jeremy',
  'bowen',
  'frontline',
  'irpin',
  'residents',
  'came',
  'russian',
  'fire',
  'trying',
  'flee']]

In [10]:
# Sözlük oluştur
dictionary = corpora.Dictionary(cleaned_texts)

# Corpus (metinlerin vektörel hali)
corpus = [dictionary.doc2bow(text) for text in cleaned_texts]

# Kontrol için ilk belgeyi yazdır
corpus[:1]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]

In [11]:
# Hücre 7: LDA Modelini Eğitme

# Eğer 'corpus' ve 'dictionary' değişkenleri tanımlanmamışsa bu hücreyi çalıştırma.
if 'corpus' in locals() and corpus and 'dictionary' in locals() and dictionary:
    print("Hücre 7 çalıştı: LDA Modeli Eğitimi.")
    # Konu sayısını belirle.
    num_topics = 5 

    print(f"\nLDA modeli {num_topics} konu ile eğitiliyor...")
    # LDA modelini eğitin (gensim.models.LdaModel kullanılıyor)
    # id2word: Sözlük
    # num_topics: Belirlenen konu sayısı
    # passes: Eğitim veri seti üzerinden kaç kez geçileceği
    # random_state: Tekrarlanabilir sonuçlar için
    # update_every: Model parametrelerinin kaç dokümanda bir güncelleneceği
    # chunksize: Her eğitim adımında kullanılacak doküman sayısı
    lda_model = LdaModel(corpus=corpus,
                         id2word=dictionary,
                         num_topics=num_topics,
                         random_state=100, 
                         update_every=1,
                         chunksize=100, 
                         passes=10, 
                         alpha='auto', 
                         per_word_topics=True) 

    print("LDA modeli eğitimi tamamlandı.")
    print("-" * 30)
else:
    print("Hata: 'corpus' veya 'dictionary' değişkeni tanımlanmamış veya boş. Lütfen 6. hücreyi kontrol edin.")


Hücre 7 çalıştı: LDA Modeli Eğitimi.

LDA modeli 5 konu ile eğitiliyor...
LDA modeli eğitimi tamamlandı.
------------------------------


In [12]:
# Hücre 8: Konuları Görüntüleme

# Eğer 'lda_model' tanımlanmamışsa bu hücreyi çalıştırma.
if 'lda_model' in locals() and lda_model:
    print("Hücre 8 çalıştı: Konuları Görüntüleme.")
    print("\n" + "="*30)
    print("LDA Modeli Konuları:")
    print("="*30)
    
    topics = lda_model.print_topics(num_words=10) 
    for idx, topic in topics:
        print(f"Konu {idx}: {topic}")
    print("="*30)
else:
    print("Hata: 'lda_model' değişkeni tanımlanmamış. Lütfen 7. hücreyi kontrol edin.")

Hücre 8 çalıştı: Konuları Görüntüleme.

LDA Modeli Konuları:
Konu 0: 0.035*"bbc" + 0.011*"man" + 0.009*"found" + 0.009*"election" + 0.009*"party" + 0.008*"family" + 0.007*"men" + 0.007*"star" + 0.006*"tells" + 0.006*"group"
Konu 1: 0.048*"says" + 0.022*"people" + 0.014*"government" + 0.014*"said" + 0.012*"president" + 0.009*"israel" + 0.008*"minister" + 0.008*"next" + 0.008*"end" + 0.007*"war"
Konu 2: 0.021*"one" + 0.014*"three" + 0.010*"made" + 0.008*"labour" + 0.008*"show" + 0.007*"woman" + 0.007*"days" + 0.007*"make" + 0.007*"number" + 0.007*"wants"
Konu 3: 0.019*"first" + 0.015*"two" + 0.014*"former" + 0.010*"win" + 0.010*"manchester" + 0.009*"day" + 0.009*"city" + 0.009*"time" + 0.009*"league" + 0.008*"trump"
Konu 4: 0.022*"year" + 0.018*"new" + 0.018*"england" + 0.018*"say" + 0.014*"police" + 0.011*"world" + 0.011*"years" + 0.011*"could" + 0.010*"old" + 0.009*"week"


In [17]:
# Hücre 9: Konu Tutarlılık (Coherence) Skorunu Hesaplama

from gensim.models.coherencemodel import CoherenceModel
if 'lda_model' in locals() and lda_model and 'cleaned_texts' in locals() and cleaned_texts and 'dictionary' in locals() and dictionary:
    print("Hücre 9 çalıştı: Konu Tutarlılık Skoru Hesaplama.")
    print("\nKonu Tutarlılık Skoru (Coherence Score) hesaplanıyor...")
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=cleaned_texts, dictionary=dictionary, coherence='c_v')

    # Coherence skorunu hesapla
    coherence_lda = coherence_model_lda.get_coherence()
    print(f'Konu Tutarlılık Skoru (Coherence Score): {coherence_lda}')
    print("="*30)
else:
     print("Hata: Gerekli değişkenler tanımlanmamış veya boş. Lütfen önceki hücreleri kontrol edin.")

Hücre 9 çalıştı: Konu Tutarlılık Skoru Hesaplama.

Konu Tutarlılık Skoru (Coherence Score) hesaplanıyor...
Konu Tutarlılık Skoru (Coherence Score): 0.25656695899335913


In [30]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from IPython.display import display
pyLDAvis.enable_notebook()

# Eğer 'lda_model', 'corpus' ve 'dictionary' tanımlıysa görselleştirme yap
if 'lda_model' in locals() and lda_model and 'corpus' in locals() and corpus and 'dictionary' in locals() and dictionary:
    print("Konu görselleştirmesi hazırlanıyor...")

    vis_data = gensimvis.prepare(lda_model, corpus, dictionary, mds='mmds')

    # HTML olarak kaydet
    pyLDAvis.save_html(vis_data, 'lda_visualization.html')
    print("Konu görselleştirmesi 'lda_visualization.html' dosyasına kaydedildi.\n")

    # Jupyter hücresinde çıktıyı göster
    display(vis_data)
else:
    print("Hata: Gerekli değişkenler tanımlanmamış veya boş.")

Konu görselleştirmesi hazırlanıyor...
Konu görselleştirmesi 'lda_visualization.html' dosyasına kaydedildi.



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.379601 -0.026812       1        1  24.810067
4      0.127631  0.324658       2        1  23.885076
1     -0.308863 -0.192128       3        1  19.649333
0     -0.271519  0.228658       4        1  16.401101
2      0.073150 -0.334376       5        1  15.254424, topic_info=          Term         Freq        Total Category  logprob  loglift
6         says  4233.000000  4233.000000  Default  30.0000  30.0000
147        bbc  2595.000000  2595.000000  Default  29.0000  29.0000
130       year  2368.000000  2368.000000  Default  28.0000  28.0000
89      people  1919.000000  1919.000000  Default  27.0000  27.0000
151      first  2061.000000  2061.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
867      house   298.433976   299.254293   Topic5  -5.4356   1.8776
617      calls   295.746327   296.578276   Topic5  -5.4446   1.8775
500   hospital   289.379242   290.199483   Topic5  -5.4664   1.8775
2309   weather   288.183180   289.017189   Topic5  -5.4705   1.8774
4718    harris   284.940479   285.773292   Topic5  -5.4818   1.8774

[181 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2270      5  0.998025  allegations
2999      3  0.998152      alleged
425       2  0.998956         also
335       3  0.998140       attack
363       1  0.998898         back
...     ...       ...          ...
28        2  0.999520        world
1097      2  0.999587        would
130       2  0.999697         year
258       2  0.999388        years
2692      5  0.998438          yet

[154 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 1, 3])